In [1]:
import lifelines as life
import pandas as pd
import numpy as np

/Users/gaoziyao/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/gaoziyao/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/gaoziyao/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# 重复事件建模的几种方法

重复事件建模需要重点考虑的是同一个对象行为的相关性。比如同一个司机的出车时长在天与天之间有高度相关性，同一个乘客打车的频次或者间隔依次是相关的。传统的生存分析方法也不适合，以为传统的生存分析考虑的是不可回溯事件，比如死亡，合约的终止等。

重复事件建模可以考虑以下集中方法：

Generalized Estimation Equation（GEE）。适用的研究问题包括司机每天出车时长（考虑为Gamma分布）；
Andersen-Gill Counting Process（AGCP)。适用于在给定时间窗内研究事件发生间隔的强度；
Prentice-Williams-Peterson Counting Process（PWPCP）。适用于研究给定时间窗内事件发生间隔的强度，且不同次序的事件发生的基本风险是不同的，因此是以事件分层的条件模型；
其他各种更加复杂的模型，略去。
这几种模型在研究的问题设定，数据要求等各方面都不一样。

GEE适合研究固定间隔时间，但是在每个固定时间窗内研究对象行为的密度。比如每个司机每天出车的时间长短即是这种特点的数据。AGCP和PWPCP都是研究counting process的模型，属于Cox模型的延伸，更加灵活，并能涵盖事后加入以及数据截断的问题。

# 对于数据组织形式的要求
因为GEE可以纳入Longitudinal study的范畴，因此这里主要讲解AGCP和PWPCP的数据组织要求。在生存分析里，时间都是相对每个研究对象的时间。

考虑下面这个问题。每个乘客从其第一次打车开始计算时间，这时的时间是0。2天后，这个乘客再次使用滴滴打车。以此类推，从第一次打车后的第4，5，10， 22， 29， 40， 70， 100天分别都使用了滴滴。由于观测窗限制到120，100天之后的第135天虽然这个乘客仍然使用了滴滴，这个数据算是截断点。AGCP和PWPCP都需要数据按照“人-时间”形式来组织，即每一次打车作为个观测数据点，包含本次打车与下次打车作为时间的起点和终点。如果在这个行为之后仍然能观测到重复发生的时间，那么截断数据标志（Event变量）就设置为1， 否则就设置为0。下面是该数据的一个展示：

In [26]:
raw = pd.read_csv('Desktop/bladder2.csv', header=0)
raw.head()

,Unnamed: 0,id,rx,number,size,start,stop,event,enum
0,1,1,1,1,3,0,1,0,1
1,2,2,1,2,1,0,4,0,1
2,3,3,1,1,1,0,7,0,1
3,4,4,1,5,1,0,10,0,1
4,5,5,1,4,1,0,6,1,1


In [27]:
raw.shape

(178, 9)

# 公式与参数解释
AGCP：
λik(t|X,β)=λ0(t)exp(Xikβ)
 
PWPCP：
λik(t|X,β)=λ0k(t)exp(Xikβ)
 
因此对于AGCP模型，Event发生的次序没有影响，而在PWPCP里面，Event发生的次序对模型有很大影响，在第一个事件发生完毕之前，是不进入第二个事件对应的风险集合，因此也被称为条件模型。比如对于乘客来说，第几次打车对于其打车的频次是有影响的，而没有打过第二次车，我们可以认为他不会打第三次车，也不会在第三次打车乘客的风险集中。

对于PWPCP， λ0k(t)λ0k(t) 是第k个事件发生的基本风险，可以想象为一个对每一个事件独有的截距项。

模型的参数解读跟Cox模型一样。这里的 βjβj 就是对应第j个自变量的hazard ratio，或者叫做相对风险。以AGCP为例，假设只有一个自变量，并且这个自变量是一个dummy variable，则取对数后：
log(λik(t|X1))=log(λ0(t))+X1×β1
 
那么对于自变量为1的人群和自变量取值为0的人群的相对风险 ϕϕ 就可以表示为：
log(ϕ(X1=1,X1=0))=log(λik(t|X1=1))−log(λik(t|X1=0))=β1
 
因此 β1β1 是相对风险在对数上的表达。

如果该变量是一个连续变量，则有：
log(ϕ(X1=a,X1=b))=log(λik(t|X1=a))−log(λik(t|X1=b))=β1×(a−b)=β1×c
 
其中， c为两类人群在这个连续数值自变量上的差。

# 如何拟合（Python和R）
对于AGCP模型，在Python中，可以使用lifelines包，并应用其针对带时间可变自变量的Cox模型拟合方法：

In [29]:
from lifelines import CoxTimeVaryingFitter

ctv = CoxTimeVaryingFitter()
ctv.fit(raw, id_col='id', event_col='event', start_col='start', stop_col='stop', show_progress=True)

Iteration 1: norm_delta = 0.734061, step_size = 0.950, ll = -458.739350, seconds_since_start = 0.2
Iteration 2: norm_delta = 0.102695, step_size = 0.950, ll = -437.185768, seconds_since_start = 0.5
Iteration 3: norm_delta = 0.005305, step_size = 0.950, ll = -436.547366, seconds_since_start = 0.7
Iteration 4: norm_delta = 0.000268, step_size = 0.950, ll = -436.545768, seconds_since_start = 0.9
Iteration 5: norm_delta = 0.000013, step_size = 0.950, ll = -436.545764, seconds_since_start = 1.1
Iteration 6: norm_delta = 0.000001, step_size = 0.950, ll = -436.545764, seconds_since_start = 1.3
Convergence completed after 6 iterations.


<lifelines.CoxTimeVaryingFitter: fitted with 178 periods, 85 uniques, 112 events>

In [36]:
print(ctv.summary)

                coef  exp(coef)  se(coef)         z             p  lower 0.95  \
Unnamed: 0 -0.002828   0.997176  0.004028 -0.702017  4.826685e-01   -0.010723   
rx         -0.039237   0.961523  0.426823 -0.091928  9.267556e-01   -0.875794   
number      0.138896   1.149005  0.049247  2.820407  4.796280e-03    0.042374   
size       -0.015367   0.984750  0.069377 -0.221507  8.246980e-01   -0.151343   
enum        0.540901   1.717554  0.101861  5.310185  1.095140e-07    0.341257   

            upper 0.95  
Unnamed: 0    0.005067  
rx            0.797320  
number        0.235418  
size          0.120608  
enum          0.740545  


在R里面，可以使用下面的命令：

In [ ]:
AGCP<-coxph(Surv(start, end, Event)~Treatment + X1 + X2 + Cluster="ID", data=raw)

对于PWPCP模型，python的lifelines包还无能为力（也许我们可以自己编写代码），但是R里面可以计算。

In [ ]:
PWPCP<-coxph(Surv(start, end, Event)~Treatment + X1 + X2 + Cluster="ID", strata='Order', data=raw)

# 总结
在滴滴的应用环境中，并没有一个真实的“生存”或者“留存”概念。实际的现象更多是需要对一种“intensity”，比如运力贡献活跃度，打车间隔密度等，的描述。在这种情况下，传统的X天留存这种二分类建模的方法并不适合。要对用户行为进行更加准确地描述的时候需要采用更为先进的模型。这里针对乘客打车间隔的密度，可以采用AGCP或者PWPCP模型进行建模，筛选影响因子。

这类模型属于Cox模型的拓展，数据构造跟带时间可变变量的Cox模型的数据结构要求一样，模型解读也一样。